In [1]:
# 2021.12.04 / KwnakiAhn / How to setup pyspark on jupyuter

# install java (https://m.blog.naver.com/opusk/220985259485)
# !sudo apt-get install default-jdk

# Install apache spark
# !wget https://spark.apache.org/downloads.html 
# (wget https://archive.apache.org/dist/spark/spark-2.3.0/spark-2.3.0-bin-hadoop2.7.tgz)
# (wget https://archive.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz) / EMR release 6.0.0
# !tar -xzf spark-1.2.0-bin-hadoop2.4.tgz
# !sudo mv spark-3.2.0-bin-hadoop3.2 /opt/spark-3.2.0
# !sudo ln -s /opt/spark-3.2.0 /opt/spark̀
# !export SPARK_HOME=/opt/spark
# !export PATH=$SPARK_HOME/bin:$PATH

# install pyspark
# !pip install pyspark
# !pip install findspark

In [2]:
import os, sys
os.environ["SPARK_HOME"] = ['/opt/spark-2.3.0', '/opt/spark-2.4.4', '/opt/spark-3.2.0'][2]
sys.path.insert(0, os.environ["SPARK_HOME"])

import findspark
findspark.init()

# 2021.12.04 / KwankiAhn / How to use graphframes in pyspark
# 1. config('spark.jars.packages') = pyspark --packages graphframes:graphframes:0.6.0-spark2.3-s_2.11
# 2. graphframes <-> spark version sensitive!
#  a. https://spark-packages.org/package/graphframes/graphframes
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("GraphFramePractice1") \
    .config("spark.some.config.option", "some-value") \
    .config('spark.jars.packages', 'graphframes:graphframes:0.8.2-spark3.2-s_2.12')\
    .getOrCreate()

:: loading settings :: url = jar:file:/opt/spark-3.2.0/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jupyter/.ivy2/cache
The jars for the packages stored in: /home/jupyter/.ivy2/jars
graphframes#graphframes added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-a94181d7-aef0-43c4-a5fc-488715841510;1.0
	confs: [default]
	found graphframes#graphframes;0.8.2-spark3.2-s_2.12 in spark-packages
	found org.slf4j#slf4j-api;1.7.16 in central
:: resolution report :: resolve 232ms :: artifacts dl 8ms
	:: modules in use:
	graphframes#graphframes;0.8.2-spark3.2-s_2.12 from spark-packages in [default]
	org.slf4j#slf4j-api;1.7.16 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	-------------------------------

In [3]:
# tutorial guide : https://graphframes.github.io/graphframes/docs/_site/quick-start.html

In [11]:
# Create a Vertex DataFrame with unique ID column "id"
sqlContext = spark
v = sqlContext.createDataFrame([
  ("a", "Alice", 34),
  ("b", "Bob", 36),
  ("c", "Charlie", 30),
], ["id", "name", "age"])
# Create an Edge DataFrame with "src" and "dst" columns
e = sqlContext.createDataFrame([
  ("a", "b", "friend"),
  ("b", "c", "follow"),
  ("c", "b", "follow"),
], ["src", "dst", "relationship"])
# Create a GraphFrame
from graphframes import *
g = GraphFrame(v, e)

# Query: Get in-degree of each vertex.
g.inDegrees.show()

# Query: Count the number of "follow" connections in the graph.
print(g.edges.filter("relationship = 'follow'").count())

# Run PageRank algorithm, and show results.
results = g.pageRank(resetProbability=0.01, maxIter=20)
results.vertices.select("id", "pagerank").show()

+---+--------+
| id|inDegree|
+---+--------+
|  b|       2|
|  c|       1|
+---+--------+

2


+---+------------------+
| id|          pagerank|
+---+------------------+
|  c|1.8994109890559092|
|  b|1.0905890109440908|
|  a|              0.01|
+---+------------------+

